In [5]:
from azureml.core import Workspace,Datastore,Dataset

In [7]:
ws = Workspace.from_config()
print(ws)

Workspace.create(name='mlws', subscription_id='d8f2241f-6180-41e3-a1de-149047238a9a', resource_group='mlrg')


In [19]:
# retrieve an existing datastore in the workspace by name
datastore  = ws.get_default_datastore()
#upload directory
dataset = Dataset.File.upload_directory(src_dir="data", 
                                        target=(datastore, "wq"))

Validating arguments.
Arguments validated.
Uploading file to winequality-red
Uploading an estimated of 2 files
Target already exists. Skipping upload for winequality-red/winequality-red.csv
Target already exists. Skipping upload for winequality-red/.ipynb_checkpoints/winequality-red-checkpoint.csv
Uploaded 0 files
Creating new dataset


In [24]:
fd = Dataset.File.from_files(path=(datastore,'/'))
for fp in fd.to_path():
    print(fp)

/winequality-red/.git/COMMIT_EDITMSG
/winequality-red/.git/FETCH_HEAD
/winequality-red/.git/HEAD
/winequality-red/.git/ORIG_HEAD
/winequality-red/.git/config
/winequality-red/.git/description
/winequality-red/.git/hooks/applypatch-msg.sample
/winequality-red/.git/hooks/commit-msg.sample
/winequality-red/.git/hooks/fsmonitor-watchman.sample
/winequality-red/.git/hooks/post-update.sample
/winequality-red/.git/hooks/pre-applypatch.sample
/winequality-red/.git/hooks/pre-commit.sample
/winequality-red/.git/hooks/pre-merge-commit.sample
/winequality-red/.git/hooks/pre-push.sample
/winequality-red/.git/hooks/pre-rebase.sample
/winequality-red/.git/hooks/pre-receive.sample
/winequality-red/.git/hooks/prepare-commit-msg.sample
/winequality-red/.git/hooks/push-to-checkout.sample
/winequality-red/.git/hooks/update.sample
/winequality-red/.git/index
/winequality-red/.git/info/exclude
/winequality-red/.git/logs/HEAD
/winequality-red/.git/logs/refs/heads/dev
/winequality-red/.git/logs/refs/remotes/o

In [26]:
# create a TabularDataset from 1 file paths in datastore
#datastore_paths = [(datastore, "winequality")]
#create a tabular object from csv
custom_ds = Dataset.Tabular.from_delimited_files(path=(datastore,"/wq/winequality-red.csv"),separator=';', header=True,encoding='utf8')


In [28]:
custom_ds.take(3).to_pandas_dataframe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5


In [29]:
#register dataset
custom_ds.register(workspace=ws,
                  name="winequality",
                  description="desc wq")

{
  "source": [
    "('workspaceblobstore', 'winequality-red/winequality-red.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "d5983c7e-aaf9-4274-8d43-f9a24cac910c",
    "name": "winequality",
    "version": 1,
    "description": "desc wq",
    "workspace": "Workspace.create(name='mlws', subscription_id='d8f2241f-6180-41e3-a1de-149047238a9a', resource_group='mlrg')"
  }
}

In [30]:
ds = Dataset.get_by_name(ws,"winequality").to_pandas_dataframe()
ds.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


In [32]:
#create train/test sets

from sklearn.model_selection import train_test_split
y = ds['quality']
X = ds.drop('quality',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [34]:
X_train.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [35]:
train_dt = X_train
train_dt['quality'] = y_train
train_dt.columns


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [36]:
test_dt = X_test
test_dt['quality'] = y_test
test_dt.columns

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [37]:
train_dt.to_csv("data/wine-quality-train.csv")
test_dt.to_csv("data/wine-quality-test.csv")

In [43]:
#datastore.upload_files(files=['data/wine-quality-train.csv','data/wine-quality-test.csv'])
#upload train & test sets to datastore
dataset2 = Dataset.File.upload_directory(src_dir="data", 
                                        target=(datastore, "wq"))

Validating arguments.
Arguments validated.
Uploading file to wq
Uploading an estimated of 4 files
Uploading data/wine-quality-test.csv
Uploaded data/wine-quality-test.csv, 1 files out of an estimated total of 4
Uploading data/wine-quality-train.csv
Uploaded data/wine-quality-train.csv, 2 files out of an estimated total of 4
Uploading data/winequality-red.csv
Uploaded data/winequality-red.csv, 3 files out of an estimated total of 4
Uploading data/.ipynb_checkpoints/winequality-red-checkpoint.csv
Uploaded data/.ipynb_checkpoints/winequality-red-checkpoint.csv, 4 files out of an estimated total of 4
Uploaded 4 files
Creating new dataset


In [47]:
trcsvpath = [(datastore,'/wq/wine-quality-train.csv')]
tstcsvpath = [(datastore,'/wq/wine-quality-test.csv')]

trds = Dataset.Tabular.from_delimited_files(path=trcsvpath)
trds = trds.register(ws,'wine-quality-train')
tstds = Dataset.Tabular.from_delimited_files(path=tstcsvpath)
tstds = trds.register(ws,'wine-quality-test')

In [50]:
Dataset.get_by_name(ws,'wine-quality-train').to_pandas_dataframe().head(3)

,Column1,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,548,12.4,0.35,0.49,2.6,0.079,27.0,69.0,0.99940,3.12,0.75,10.4,6
1,355,6.7,0.75,0.01,2.4,0.078,17.0,32.0,0.99550,3.55,0.61,12.8,6
2,1296,6.6,0.63,0.00,4.3,0.093,51.0,NaN,0.99558,3.20,0.45,9.5,5
